In [ ]:
# import libraries
from urllib.request import urlopen
from selenium import webdriver
from bs4 import BeautifulSoup
from multiprocessing import Pool
import csv
import requests

In [ ]:
data2 = [] # set up initial data list for IMDB urls
nndb_names = [] # set up initial name list to make NNDB search urls
url_list = [] # set up initial url list for NNDB search urls
link_list = [] # set up initial link list for NNDB actor urls 
race_list = [] # set up initial race list for actors' races

url = "http://www.imdb.com/name/" # set up base url for IMDB search
query_url = "http://search.nndb.com/search/nndb.cgi?type=unspecified&query=" # set up base url for NNDB search

In [ ]:
# function to search actor via name ID (nm###...) and store actual name 
# ...takes in specific url and scrapes it

def scrapeIMDB(url):
    name_list = []
    
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")
        page.close()

        actor = soup.find("span", attrs = {"itemprop": "name"}) # find & store actor's actual name
        img_url = soup.find("img", attrs = {"id": "name-poster"})
        if img_url != None:
            img_url = img_url.get("src")
            print(url[-9:] + "," + actor.text.strip() + "," + img_url) # print the actor's name id with their actual name in 'nm####,name' format for easy csv parsing printing is important because pooling always ends in error
        else:
            img_url = "NA"
            print(url[-9:] + "," + actor.text.strip() + "," + img_url)  
                                                                       

        return name_list
    except Exception as e: 
        print("shit stopped working at url: " + url) # exception for cases where for some reason url invalid 
        raise e

In [ ]:
with open('Documents/leftover_ids.csv', newline='') as csvfile: # open character id file
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|') # read it in, quotechar unnecessary
    for row in reader:
        data2.append(url + row[0]) # for each row in the csv, append the value in the first column to the end 
                                  # of the IMDB url, creating a list of accessible IMDB urls

In [ ]:
list_size = 3000 # declare rough size of list to deal with 

num_workers = 4 * 2 # should be using the number of cpu cores in the computer via 
                    # some reference, but macbooks have 4 cores, so this works
    
chunksize = list_size // num_workers * 4  # Try to get a good chunksize. You're probably going to have 
                                          # to tweak this, though. Try smaller and lower values and see 
                                          # how performance changes.
        
pool = Pool(num_workers) # don't fully understand this call, but it allows the pool.map function to run

actors = pool.map(scrapeIMDB, data2) # calls function 'scrapeIMDB' using elements of list 'data'
                                    # won't end properly, but that's why we print the output
    
# COPY AND PASTE OUTPUT into Actors.csv

In [ ]:
# function takes in NNDB search url, and if search was a match, records the
# first link in the database (to the matched name). If no match, records NA value

def scrapeLink(url):
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")
        page.close()

        global link_list # calling in link_list in order to modify, but hasn't worked yet
        temp = soup.find("p")
        
        if temp is not None: # must check if not none even though you have try/exception — makes sure you 
                             # perform function on empty value 
            
            if soup.find("p").contents[0] == '\nNo names match your query ':
                link_list.append("NA")
                print(url + "," + "NA") # print the url in order to record characters not on NNDB 
            else:
                link = soup.findAll("a", href = True)[2]['href'] # find link to actor's actual NNDB page
                link_list.append(link) # append link to link_list

                print(url + "," + link) # print url in order to stay consistent
        else:
            print(url + "," + "NA") # again, print url to record which links don't work

        return link_list
    except Exception as e:
        print("shit stopped working at url: " + url)
        raise e

In [ ]:
def scrapeRace(url):
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")
        page.close()

        global race_list # global to preserve modification outside of function

        name = soup.find("font", attrs = {"size": "+3"}) # get name of actor to tie name to race
        race = soup.find("b", text="Race or Ethnicity:") # get field for race of actor
        
        if name is not None and race is not None:
            name = name.text.strip() # get name in string form
            race = race.next_sibling # get race of actor
            race_list.append(race) # append race to race_list

            print(name + "," + race) # print name and race to log race of each actor in csv parseable format

        return race_list
    
    except Exception as e:
        print("shit stopped working at url: " + url)
        raise e

In [ ]:
link_list

In [ ]:
# need each individual word separate for NNDB url
with open('Downloads/Actors.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        nndb_names.append(row[1:])

In [ ]:
# removing empty list items so url structure makes sense 
# (e.g. "...michael+keaton" instead of "...michael+keaton+")

for i in range(0, len(nndb_names)):
    nndb_names[i] = list(filter(None, nndb_names[i]))

In [ ]:
for i in range (0, len(nndb_names)): # cycle through the entire list of actors to search on NNDB
    
    for j in range(0, len(nndb_names[i])): # search through list of first, middle, & last names for each actor
        
        if len(nndb_names[i]) == 3 and j == 0: # if we're at the first name of a person with f, m, & l
            page3 = query_url + nndb_names[i][j] + "+" + nndb_names[i][j+1] + "+"  # add first two names to url +
            
        elif j == 0: # if we're at the first name of a person with just f & l
            page3 = query_url + nndb_names[i][j] + "+" # add first name +
            
        elif j == len(nndb_names[i])-1: # if we're a the last name of a person
            page3 = page3 + nndb_names[i][j] # add their last name to the url
            
    url_list.append(page3) # append completed urls to list of searchable NNDB query urls
    page3 = "" # set page equal to none again since every name ends with a full page3 value

In [ ]:
for idx, item in enumerate(url_list): # this is to remove pesky characters that somehow enter the fold
                                      # and fuck with your program — it replaces any url containing one 
                                      # of these characters with a searchable url that will return an NA value
    if 'â' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '€' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '¦' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif 'Ã' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '¥' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '«' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '¢' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif '¾' in item:
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Dennis+North'
    elif item == 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Michael+PeÃ‘a':
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Michael+Pena'
    elif item == 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Ren_e+Zellweger':
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Renee+Zellweger'
    elif item == 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=RZA+':
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=RZA'
    elif item == 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Juan+Carlos+Hernâ€¦ndez':
        url_list[idx] = 'http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Juan+Carlos+Hernandez'

In [ ]:
list_size = 33944 
num_workers = 4 * 2
chunksize = list_size // num_workers * 5
pool = Pool(num_workers)

links = pool.map(scrapeLink, url_list)

# COPY AND PASTE OUTPUT into NNDB_Actors.csv (should change name to something more accurate, like NNDB_links)

In [ ]:
with open('Documents/NNDB_Actors.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        link_list.append(row[1])

In [ ]:
list_size = 12000 
num_workers = 4 * 2
chunksize = list_size // num_workers * 4

pool2 = Pool(num_workers)
races = pool2.map(scrapeRace, link_list)

In [ ]:
# function to search actor via name ID (nm###...) and store actual name 
# ...takes in specific url and scrapes it

def scrapeInfo(url):
    try:
        page = urlopen(url)
        soup = BeautifulSoup(page, "html.parser")
        page.close()

        info_list = []
        
        gross = soup.find("h4", text = "Gross USA:")
        if (gross != None):
            gross = gross.next_sibling
        production = soup.find("h4", text = "Production Co:")
        if (production != None):
            production = production.next_sibling.next_sibling.text.strip()
        genre = soup.find("div", attrs = {"itemprop": "genre"}) 
        if (genre != None):
            genre = genre.text.strip()
            
        if (gross != None and production != None and genre != None):
            print(url[-9:] + "," + gross.encode("unicode_escape").decode("utf-8") + "," + production.encode("unicode_escape").decode("utf-8") + "," + genre.encode("unicode_escape").decode("utf-8")) # print the actor's name id with their actual name in
                                                  

        return info_list
    except Exception as e: 
        print("shit stopped working at url: " + url) # exception for cases where for some reason url invalid 
        raise e

In [ ]:
list_size = 25000 
num_workers = 4 * 2
chunksize = list_size // num_workers * 4

pool2 = Pool(num_workers)
races = pool2.map(scrapeInfo, title_list)

In [ ]:
url = "http://www.imdb.com/title/"

In [ ]:
title_list = []

with open('Documents/TitleList.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        title_list.append(url + row[1])

In [ ]:
title_list

In [ ]:
genre

In [ ]:
url_list

In [ ]:
temp = []

for i in range (0, 3):
    page2 = urlopen(url_list[i])
    soup2 = BeautifulSoup(page2, "html.parser")
    page2.close()

    if soup2.find("p").contents[0] == '\nNo names match your query ':
        temp.append("NA")
    else:
        link = soup2.findAll("a", href = True)[2]['href']
        temp.append(link)
    

In [ ]:
race_list = []

for i in range (0, len(temp)):
    if temp[i] != "NA":
        page5 = urlopen(temp[i])
        soup5 = BeautifulSoup(page5, "html.parser")
        page5.close()
        
        name = soup5.find("font", attrs = {"size": "+3"}).text.strip()
        race = soup5.find("b", text="Race or Ethnicity:").next_sibling
        race_list.append(race)
    else: 
        race_list.append("NA")

In [ ]:
chunksize

In [ ]:
nndb_names

In [ ]:
ulimit -n

In [ ]:
text.contents[0]

In [ ]:
page5 = urlopen('http://search.nndb.com/search/nndb.cgi?type=unspecified&query=Waldemar+Kobus')
soup5 = BeautifulSoup(page5, "html.parser")
page5.close()
        
race = soup5.find("k")

In [ ]:
link_list[-0]

In [ ]:
with open('Documents/Actors.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(["name"]) # write csv file headers
    for i in range (0, len(actors)):
        writer.writerow([actors[i]])